In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load data

In [5]:
import numpy as np
import keras


# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()


x_train = np.expand_dims(x_train, axis=-1) / 255.
x_test = np.expand_dims(x_test, axis=-1) / 255.

print(f"{x_train.shape=} {y_train.shape=}")
print(f"{x_train.shape=} {y_test.shape=}")


ModuleNotFoundError: No module named 'tensorflow'

## Visualise Data

### Label distribution

In [ ]:
from collections import Counter
from matplotlib import pyplot as plt

label_distribution = Counter(y_train)
plt.bar(label_distribution.keys(), label_distribution.values())
plt.show()